### No Noise Data

#### No Noise Data (Training)

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
from collections import deque
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import numpy as np

In [2]:
file_path = 'cf_train_no_noise.csv'
file_path_test = 'cf_test_no_noise.csv'
data = pd.read_csv(file_path)
data_test = pd.read_csv(file_path_test)

In [3]:
unnecessary_columns = ['row_num', 'day', 'era', 'target_10_val', 'target_5_val', 'sigma', 'day_no']
# Train
X_train = data.drop(columns=unnecessary_columns)
y_train = data['target_10_val']
y_train = to_categorical((y_train*4).astype(int))
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Test
X_test = data_test.drop(columns=unnecessary_columns)
X_test = scaler.transform(X_test)
y_test = to_categorical((data_test['target_10_val']*4).astype(int))

In [4]:
class ConceptDriftDetector:
    def __init__(self, threshold=0.05, buffer_size=50):
        self.threshold = threshold
        self.buffer_size = buffer_size
        self.errors = deque(maxlen=buffer_size)
        self.previous_error_rate = None

    def detect(self, y_trues, y_preds):
        y_trues = np.array(y_trues).flatten()
        y_preds = np.array(y_preds).flatten()
        batch_errors = [int(y_true != y_pred) for y_true, y_pred in zip(y_trues, y_preds)]
        self.errors.extend(batch_errors)
        if len(self.errors) == self.buffer_size:
            error_rate = sum(self.errors) / self.buffer_size
            if self.previous_error_rate is not None:
                if error_rate - self.previous_error_rate > self.threshold:
                    print(f"Concept drift detected: Error rate increased from {self.previous_error_rate} to {error_rate}")
                    return True
            self.previous_error_rate = error_rate
        return False

In [5]:
drift_detector = ConceptDriftDetector(threshold=0.04, buffer_size=50)

In [6]:
class SlidingWindowKNN:
    def __init__(self, window_size, n_neighbors):
        self.window_size = window_size
        self.n_neighbors = n_neighbors
        self.scaler = StandardScaler()
        self.knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        self.window = deque(maxlen=window_size)
    def fit(self, X, y):
        X_scaled = self.scaler.fit_transform(X)
        self.knn.fit(X_scaled, y)
    def partial_fit(self, X_new, y_new):
        self.window.append((X_new, y_new))
        if len(self.window) == self.window_size:
            X_window, y_window = zip(*self.window)
            X_window_scaled = self.scaler.transform(X_window)
            self.knn.fit(X_window_scaled, y_window)
    def predict(self, X):
        X_scaled = self.scaler.transform(X)
        return self.knn.predict(X_scaled)

In [7]:
window_size = 1000
n_neighbors = 15
model = SlidingWindowKNN(window_size, n_neighbors)
model.fit(X_train[:window_size], y_train[:window_size])

In [8]:
step_size = 100
for i in tqdm(range(window_size, len(X_train), step_size)):
    X_new = X_train[i:i+step_size][0]  
    y_new = y_train[i:i+step_size][0]
    model.partial_fit(X_new, y_new)
    y_pred = model.predict([X_new])

    if i % 10000 == 0:
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f'Accuracy at {i} is {accuracy*100}%')
    
    drift = drift_detector.detect(y_trues=y_new, y_preds=y_pred)

    if drift:
        model = SlidingWindowKNN(window_size, n_neighbors)
        model.fit(X_train[max(0, i-window_size):i], y_train[max(0, i-window_size):i])

  0%|          | 0/614 [00:00<?, ?it/s]

 10%|█         | 62/614 [00:00<00:02, 242.60it/s]

Concept drift detected: Error rate increased from 0.12 to 0.16


 28%|██▊       | 170/614 [00:02<00:06, 63.74it/s]

Accuracy at 10000 is 68.59455128205128%
Concept drift detected: Error rate increased from 0.06 to 0.1
Concept drift detected: Error rate increased from 0.06 to 0.1
Concept drift detected: Error rate increased from 0.06 to 0.1
Concept drift detected: Error rate increased from 0.06 to 0.1
Concept drift detected: Error rate increased from 0.06 to 0.1


...

Accuracy at 40000 is 69.55608974358974%


 80%|███████▉  | 491/614 [00:13<00:03, 36.74it/s]

Accuracy at 50000 is 69.55608974358974%
Concept drift detected: Error rate increased from 0.06 to 0.1
Concept drift detected: Error rate increased from 0.06 to 0.1
Concept drift detected: Error rate increased from 0.06 to 0.1


100%|██████████| 614/614 [00:15<00:00, 39.15it/s]

Accuracy at 60000 is 70.50160256410255%
Concept drift detected: Error rate increased from 0.1 to 0.14


#### No Noise Data (Results)

In [9]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Testing Accuracy: {accuracy * 100:.2f}%")

Testing Accuracy: 70.22%


### Noisy data

#### Noisy data (Training)

In [17]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
from collections import deque
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [39]:
file_path = 'cf_train.csv'
file_path_test = 'cf_test.csv'
data = pd.read_csv(file_path)
data_test = pd.read_csv(file_path_test)

In [40]:
unnecessary_columns = ['row_num', 'day', 'era', 'target_10_val', 'target_5_val', 'sigma', 'day_no']
# Train
X_train = data.drop(columns=unnecessary_columns)
y_train = data['target_10_val']
y_train = to_categorical((y_train*4).astype(int))
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Test
X_test = data_test.drop(columns=unnecessary_columns)
X_test = scaler.transform(X_test)
y_test = to_categorical((data_test['target_10_val']*4).astype(int))

In [41]:
class SlidingWindowKNN:
    def __init__(self, window_size, n_neighbors):
        self.window_size = window_size
        self.n_neighbors = n_neighbors
        self.scaler = StandardScaler()
        self.knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        self.window = deque(maxlen=window_size)
    
    def fit(self, X, y):
        X_scaled = self.scaler.fit_transform(X)
        self.knn.fit(X_scaled, y)
    
    def partial_fit(self, X_new, y_new):
        self.window.append((X_new, y_new))
        if len(self.window) == self.window_size:
            X_window, y_window = zip(*self.window)
            X_window_scaled = self.scaler.transform(X_window)
            self.knn.fit(X_window_scaled, y_window)
    
    def predict(self, X):
        X_scaled = self.scaler.transform(X)
        return self.knn.predict(X_scaled)

In [42]:
class ConceptDriftDetector:
    def __init__(self, threshold=0.05, buffer_size=50):
        self.threshold = threshold
        self.buffer_size = buffer_size
        self.errors = deque(maxlen=buffer_size)
        self.previous_error_rate = None

    def detect(self, y_trues, y_preds):
        y_trues = np.array(y_trues).flatten()
        y_preds = np.array(y_preds).flatten()
        batch_errors = [int(y_true != y_pred) for y_true, y_pred in zip(y_trues, y_preds)]
        self.errors.extend(batch_errors)
        if len(self.errors) == self.buffer_size:
            error_rate = sum(self.errors) / self.buffer_size
            if self.previous_error_rate is not None:
                if error_rate - self.previous_error_rate > self.threshold:
                    print(f"Concept drift detected: Error rate increased from {self.previous_error_rate} to {error_rate}")
                    return True
            self.previous_error_rate = error_rate
        return False

In [43]:
drift_detector = ConceptDriftDetector(threshold=0.04, buffer_size=50)

In [44]:
window_size = 1000
n_neighbors = 15
model = SlidingWindowKNN(window_size, n_neighbors)
model.fit(X_train[:window_size], y_train[:window_size])

In [45]:
step_size = 100
for i in tqdm(range(window_size, len(X_train), step_size)):
    X_new = X_train[i:i+step_size][0]  
    y_new = y_train[i:i+step_size][0]
    model.partial_fit(X_new, y_new)
    y_pred = model.predict([X_new])

    if i % 10000 == 0:
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f'Accuracy at {i} is {accuracy*100}%')
    
    drift = drift_detector.detect(y_trues=y_new, y_preds=y_pred)

    if drift:
        model = SlidingWindowKNN(window_size, n_neighbors)
        model.fit(X_train[max(0, i-window_size):i], y_train[max(0, i-window_size):i])

  0%|          | 0/614 [00:00<?, ?it/s]

Concept drift detected: Error rate increased from 0.18 to 0.22
Concept drift detected: Error rate increased from 0.18 to 0.22
Concept drift detected: Error rate increased from 0.18 to 0.24
Concept drift detected: Error rate increased from 0.18 to 0.24
Concept drift detected: Error rate increased from 0.18 to 0.24
Concept drift detected: Error rate increased from 0.18 to 0.26
Concept drift detected: Error rate increased from 0.18 to 0.26
Concept drift detected: Error rate increased from 0.18 to 0.22
Concept drift detected: Error rate increased from 0.18 to 0.22
Concept drift detected: Error rate increased from 0.12 to 0.16
Concept drift detected: Error rate increased from 0.12 to 0.16
Concept drift detected: Error rate increased from 0.12 to 0.16
Concept drift detected: Error rate increased from 0.16 to 0.2
Concept drift detected: Error rate increased from 0.16 to 0.22


 28%|██▊       | 174/614 [00:02<00:05, 76.84it/s]

Accuracy at 10000 is 26.991987179487182%
Concept drift detected: Error rate increased from 0.16 to 0.24
Concept drift detected: Error rate increased from 0.16 to 0.22
Concept drift detected: Error rate increased from 0.16 to 0.22
Concept drift detected: Error rate increased from 0.16 to 0.2
Concept drift detected: Error rate increased from 0.16 to 0.2
Concept drift detected: Error rate increased from 0.16 to 0.22
Concept drift detected: Error rate increased from 0.16 to 0.24
Concept drift detected: Error rate increased from 0.16 to 0.24
Concept drift detected: Error rate increased from 0.16 to 0.22
Concept drift detected: Error rate increased from 0.16 to 0.22
Concept drift detected: Error rate increased from 0.16 to 0.2
Concept drift detected: Error rate increased from 0.16 to 0.24
Concept drift detected: Error rate increased from 0.16 to 0.22
Concept drift detected: Error rate increased from 0.16 to 0.22
Concept drift detected: Error rate increased from 0.16 to 0.2
Concept drift dete

 37%|███▋      | 225/614 [00:05<00:09, 39.46it/s]

...

Accuracy at 40000 is 26.150641025641026%
Concept drift detected: Error rate increased from 0.1 to 0.18
Concept drift detected: Error rate increased from 0.1 to 0.16
Concept drift detected: Error rate increased from 0.1 to 0.18
Concept drift detected: Error rate increased from 0.1 to 0.18
Concept drift detected: Error rate increased from 0.1 to 0.18
Concept drift detected: Error rate increased from 0.1 to 0.16
Concept drift detected: Error rate increased from 0.1 to 0.14
Concept drift detected: Error rate increased from 0.1 to 0.14
Concept drift detected: Error rate increased from 0.1 to 0.16
Concept drift detected: Error rate increased from 0.1 to 0.16
Concept drift detected: Error rate increased from 0.1 to 0.16
Concept drift detected: Error rate increased from 0.1 to 0.2
Concept drift detected: Error rate increased from 0.1 to 0.22
Concept drift detected: Error rate increased from 0.1 to 0.22
Concept drift detected: Error rate increased from 0.1 to 0.22
Concept drift detected: Error 

 95%|█████████▍| 582/614 [00:13<00:00, 49.37it/s]

Accuracy at 50000 is 25.560897435897434%
Concept drift detected: Error rate increased from 0.1 to 0.14
Concept drift detected: Error rate increased from 0.12 to 0.16
Concept drift detected: Error rate increased from 0.12 to 0.2
Concept drift detected: Error rate increased from 0.12 to 0.22
Concept drift detected: Error rate increased from 0.12 to 0.22
Concept drift detected: Error rate increased from 0.12 to 0.22
Concept drift detected: Error rate increased from 0.12 to 0.18
Concept drift detected: Error rate increased from 0.12 to 0.16
Concept drift detected: Error rate increased from 0.12 to 0.2
Concept drift detected: Error rate increased from 0.12 to 0.22
Concept drift detected: Error rate increased from 0.12 to 0.26
Concept drift detected: Error rate increased from 0.12 to 0.26
Concept drift detected: Error rate increased from 0.12 to 0.26
Concept drift detected: Error rate increased from 0.12 to 0.28
Concept drift detected: Error rate increased from 0.12 to 0.26
Concept drift det

100%|██████████| 614/614 [00:15<00:00, 38.76it/s]

Accuracy at 60000 is 23.825320512820515%
Concept drift detected: Error rate increased from 0.12 to 0.24
Concept drift detected: Error rate increased from 0.12 to 0.24
Concept drift detected: Error rate increased from 0.12 to 0.2
Concept drift detected: Error rate increased from 0.12 to 0.2
Concept drift detected: Error rate increased from 0.12 to 0.18
Concept drift detected: Error rate increased from 0.12 to 0.18
Concept drift detected: Error rate increased from 0.12 to 0.22
Concept drift detected: Error rate increased from 0.12 to 0.22
Concept drift detected: Error rate increased from 0.12 to 0.2
Concept drift detected: Error rate increased from 0.12 to 0.22
Concept drift detected: Error rate increased from 0.12 to 0.2
Concept drift detected: Error rate increased from 0.12 to 0.18
Concept drift detected: Error rate increased from 0.12 to 0.22
Concept drift detected: Error rate increased from 0.12 to 0.22
Concept drift detected: Error rate increased from 0.12 to 0.22
Concept drift dete

#### Noisy date (Results)

In [46]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Testing Accuracy: {accuracy * 100:.2f}%")

Testing Accuracy: 22.71%
